# 1. Preprocess-Stage1-Group-All-TrainData

## Run name

In [1]:
import time
project_name = 'Google-LandMark-Rec2019'
step_name = '1-Preprocess-Stage1-Group-All-TrainData'
time_str = time.strftime("%Y%m%d-%H%M%S", time.localtime())
run_name = project_name + '_' + step_name + '_' + time_str
print('run_name: ' + run_name)
t0 = time.time()

run_name: Google-LandMark-Rec2019_1-Preprocess-Stage1-Group-All-TrainData_20190503-052200


## Important params

In [2]:
import multiprocessing

cpu_amount = multiprocessing.cpu_count()
# train_tar_count = 500
train_data_ratio = 0.95

# print('train_tar_count:', train_tar_count)
print('cpu_amount: ', cpu_amount)
print(train_data_ratio)

cpu_amount:  2
0.95


## Import PKGs

In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
%matplotlib inline
from IPython.display import display
from sklearn.model_selection import train_test_split

import os, sys, gc, math, shutil, zipfile, pickle, h5py, re, tarfile
import urllib, xlsxwriter
from tqdm import tqdm
import hashlib

## Project folders

In [4]:
cwd = os.getcwd()
feature_folder = os.path.join(cwd, 'feature')
input_folder = os.path.join(cwd, 'input')
output_folder = os.path.join(cwd, 'output')
model_folder = os.path.join(cwd, 'model')

md5sum_folder = os.path.join(input_folder, 'md5sum')
if not os.path.exists(md5sum_folder):
    os.mkdir(md5sum_folder)
    print('create folder:', md5sum_folder)
    
train_tar_folder = os.path.join(input_folder, 'train_tar')
if not os.path.exists(train_tar_folder):
    os.mkdir(train_tar_folder)
    print('create folder:', train_tar_folder)
    
train_untar_folder = os.path.join(input_folder, 'train_untar')
if not os.path.exists(train_untar_folder):
    os.mkdir(train_untar_folder)
    print('create folder:', train_untar_folder)
    
all_image_folder = os.path.join(input_folder, 'all_image')
if not os.path.exists(all_image_folder):
    os.mkdir(all_image_folder)
    print('create folder:', all_image_folder)
    
data_train_folder = os.path.join(input_folder, 'data_train')
if not os.path.exists(data_train_folder):
    os.mkdir(data_train_folder)
    print('create folder:', data_train_folder)
    
data_val_folder = os.path.join(input_folder, 'data_val')
if not os.path.exists(data_val_folder):
    os.mkdir(data_val_folder)
    print('create folder:', data_val_folder)

create folder: /data/landmark-recognition-2019/input/data_train
create folder: /data/landmark-recognition-2019/input/data_val


In [5]:
!pwd
!ls ./input -hl

/data/landmark-recognition-2019
total 1.7G
drwxr-xr-x   2 root   root   160M May  2 21:14 all_image
drwxr-xr-x   2 root   root   4.0K May  3 05:22 data_train
drwxr-xr-x   2 root   root   4.0K May  3 05:22 data_val
-rw-rw-r--   1 ubuntu ubuntu 108M May  1 17:18 google-landmarks-dataset.zip
drwxr-xr-x   2 root   root    24K May  1 17:29 md5sum
----------   1 root   root   3.0M Apr  8 02:17 recognition_sample_submission.csv
-rw-r--r--   1 root   root   1.3M May  3 05:08 recognition_sample_submission.csv.zip
-rw-r--r--   1 root   root   965M Apr  5 21:54 train_attribution.csv
-rw-r--r--   1 root   root   502M Apr  5 21:57 train.csv
drwxr-xr-x   2 root   root    20K May  2 20:39 train_tar
drwxr-xr-x 502 root   root    20K May  2 20:39 train_untar


In [6]:
!ls ./input/all_image/ -U|head -5

e42936e3627d8989.jpg
96d0f4fec18d8c84.jpg
02fd1c4af95765fd.jpg
37bdd02aebba82a6.jpg
b732241b0be6a043.jpg
ls: write error: Broken pipe


In [7]:
# !wget https://s3.amazonaws.com/google-landmark/metadata/train.csv
# !wget https://s3.amazonaws.com/google-landmark/metadata/train_attribution.csv

In [8]:
train_csv = os.path.join(input_folder, 'train.csv')
test_csv = os.path.join(input_folder, 'test.csv')
boxes_split1_csv = os.path.join(input_folder, 'google_landmark_boxes_split1.csv')
boxes_split2_csv = os.path.join(input_folder, 'google_landmark_boxes_split2.csv')
recognition_sample_submission_csv = os.path.join(input_folder, 'recognition_sample_submission.csv')

print(train_csv)
print(test_csv)
print(boxes_split1_csv)
print(boxes_split2_csv)
print(recognition_sample_submission_csv)


/data/landmark-recognition-2019/input/train.csv
/data/landmark-recognition-2019/input/test.csv
/data/landmark-recognition-2019/input/google_landmark_boxes_split1.csv
/data/landmark-recognition-2019/input/google_landmark_boxes_split2.csv
/data/landmark-recognition-2019/input/recognition_sample_submission.csv


## Preview csv

In [9]:
%%time
train_df = pd.read_csv(train_csv, engine='python', nrows=None)
display(train_df.shape, train_df.head(2))

(4132914, 3)

,id,url,landmark_id
0,6e158a47eb2ca3f6,https://upload.wikimedia.org/wikipedia/commons...,142820
1,202cd79556f30760,http://upload.wikimedia.org/wikipedia/commons/...,104169


CPU times: user 17.4 s, sys: 1.09 s, total: 18.5 s
Wall time: 18.3 s


In [10]:
data_train_df, data_val_df = train_test_split(train_df, train_size=train_data_ratio)
display(data_train_df.shape, data_train_df.head(2))
display(data_val_df.shape, data_val_df.head(2))

/root/anaconda3/lib/python3.6/site-packages/sklearn/model_selection/_split.py:2026: FutureWarning: From version 0.21, test_size will always complement train_size unless both are specified.
  FutureWarning)


(3926268, 3)

,id,url,landmark_id
210540,cbb444c0030bfa4b,https://upload.wikimedia.org/wikipedia/commons...,108049
2860659,4d646b4d708774ac,https://upload.wikimedia.org/wikipedia/commons...,144088


(206646, 3)

,id,url,landmark_id
1110858,7de387b44621a7af,https://upload.wikimedia.org/wikipedia/commons...,176343
1756123,bea8782edec2c797,http://upload.wikimedia.org/wikipedia/commons/...,39511


## Group all categories by creating soft link

In [11]:
%%time
# data_train
data_train_invalid_count = 0
data_train_softlink_count = 0
for idx, row in data_train_df.iterrows():
    src_image_path = os.path.join(all_image_folder, '%s.jpg' % row['id'])
#     print(src_image_path)
    if not os.path.exists(src_image_path):
        data_train_invalid_count += 1
        continue
#     print(src_image_path)
    target_image_folder = os.path.join(data_train_folder, '%06d' % row['landmark_id'])
#     print(target_image_folder)
    if not os.path.exists(target_image_folder):
        os.mkdir(target_image_folder)
#         print('Folder created:', target_image_folder)
    target_image_path = os.path.join(target_image_folder, '%s.jpg' % row['id'])
    if not os.path.exists(target_image_path):
        data_train_softlink_count += 1
        os.symlink(src_image_path, target_image_path)
    if (idx+1) % 10000 == 0:
        print('*', end='')
    if (idx+1) % 1000000 == 0:
        print('|')

print('data_train_invalid_count:', data_train_invalid_count)
print('data_train_softlink_count:', data_train_softlink_count)

**************************************************************************************************************************************************************************|
*********************************************************************************|
******************************************************************************************************************|
**********************data_train_invalid_count: 0
data_train_softlink_count: 3926268
CPU times: user 16min 41s, sys: 5min 38s, total: 22min 20s
Wall time: 4h 13min 19s


In [12]:
%%time
# data_val
data_val_invalid_count = 0
data_val_softlink_count = 0
for idx, row in data_val_df.iterrows():
    src_image_path = os.path.join(all_image_folder, '%s.jpg' % row['id'])
#     print(src_image_path)
    if not os.path.exists(src_image_path):
        data_val_invalid_count += 1
        continue
#     print(src_image_path)
    target_image_folder = os.path.join(data_val_folder, '%06d' % row['landmark_id'])
#     print(target_image_folder)
    if not os.path.exists(target_image_folder):
        os.mkdir(target_image_folder)
#         print('Folder created:', target_image_folder)
    target_image_path = os.path.join(target_image_folder, '%s.jpg' % row['id'])
    if not os.path.exists(target_image_path):
        data_val_softlink_count += 1
        os.symlink(src_image_path, target_image_path)
    if (idx+1) % 10000 == 0:
        print('*', end='')
    if (idx+1) % 100000 == 0:
        print('|')

print('data_val_invalid_count:', data_val_invalid_count)
print('data_val_softlink_count:', data_val_softlink_count)

**********|
**************|
**data_val_invalid_count: 0
data_val_softlink_count: 206646
CPU times: user 47.9 s, sys: 21.5 s, total: 1min 9s
Wall time: 10min 4s


In [13]:
print('Time elapsed: %.1fs' % (time.time() - t0))
print(run_name)

Time elapsed: 15826.4s
Google-LandMark-Rec2019_1-Preprocess-Stage1-Group-All-TrainData_20190503-052200
